# Gradient checking 

* Gradient checking is an effective tool to make sure the proper working of backward propagation. 
* We are only checking wokring of backward propagation since we are confident that our forward propagation implementation works fine. 
* First, we will implement 1-d gradient checking so as to build an intuitions about it then we will implement N-dimentional gradient checking.
* This implementation is a three layer implementation with output layer having sigmoid activation and other layers having ReLu activation. 
* Network parameters (input = 4, 5, 3, output = 1)
* Loss: cross entropy loss

## understanding gradient checking process: 
* When we increase and decrease the same parameter values by epsilon, then we only calculate the gradapprox for only that parameter.
* This means that, for each paramerters, we need to impelment complete forward propagation with thetaplus and again with thetaminus and then calculate the gradapprox value for that particular parameter. 
* For calculating gradappox of one parameter, we implement forward propagation twice, so for complete evaluation, we need to implement 2*number_of_parameters time forward propagation. 
* This requirememtn makes it costly to calculate, so this should not be done at each step.

In [1]:
# imports 
import numpy as np 
from testCases import *

# Supporting functions 
* There are certain function that we need to calculate forward propagation and backward propagation

In [2]:
# Sigmoid
def sigmoid(x): 
    """
    calculates sigmoid of x.
    
    Argument: 
    x : A scalar or numpy array of any size. 
    
    Return: 
    Sig: sigmoid value of array or scalar x.
    """
    return 1/(1 + np.exp(-x))

# Relu 
def relu(x): 
    """
    Calculates the ReLu of x
    
    Argument: 
    x : a scalar of numpy array of any size. 
    
    Return: 
    relu: returns the relu(x)
    """
    return np.maximum(0,x)

* Druing the calculation of derivative, we need to convert all parameters into on giant vector, which will be implemented by dictionary_to_vector function. We would also need to revert the vector back into its original form, for that we will be using vector_to_dictionary function. 

In [63]:
# dictionary to vector.
def dictionary_to_vector(parameters): 
    """
    Roll all parameters into a single vector.
    
    Argument: 
    parameters: a dictionary of parameters 
    
    Return: 
    theta: a column vector comprising of all parameters stacked.
    """
    index = [0]
    initial_index = 0
    count =0
    for key in parameters.keys(): 
        new_vector = np.reshape(parameters[key], (-1,1))
        end_index = index[-1] + new_vector.shape[0]
        index.append(end_index)
        if count == 0: 
            theta = new_vector
        else: 
            theta = np.concatenate((theta, new_vector), axis=0)
        count +=1
    return theta, index

# vector to dictionary 
def vector_to_dictionary(theta, index, param):
    
    parameters = {}
    parameters["W1"] = theta[:index[1]].reshape(param["W1"].shape)
    parameters["b1"] = theta[index[1]:index[2]].reshape(param["b1"].shape)
    parameters["W2"] = theta[index[2]:index[3]].reshape(param["W2"].shape)
    parameters["b2"] = theta[index[3]:index[4]].reshape(param["b2"].shape)
    parameters["W3"] = theta[index[4]:index[5]].reshape(param["W3"].shape)
    parameters["b3"] = theta[index[5]:index[6]].reshape(param["b3"].shape)
    
    return parameters

# gradients to vector 
def gradients_to_vector(gradients):
    """
    Roll all our gradients dictionary into a single vector.
    """
    
    count = 0
    for key in ["dW1", "db1", "dW2", "db2", "dW3", "db3"]:
        # flatten parameter
        new_vector = np.reshape(gradients[key], (-1,1))
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count = count + 1

    return theta
    
    

In [60]:
# N-D implementation to understand the steps involved 

# forward propagation
def forward_propagation(X, Y, parameters): 
    """
    Implements the forward propagation and computes cost.
    
    Arguments: 
    X : traininig set examples 
    Y : true labels 
    parameters : dictionary of parameters W and b.
    
    Return: 
    cost: total cross entropy cost 
    cache: cache of parameters,pre-activation, activation and bias for all layers.
    """
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    # forward step 
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    # cost 
    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1-A3), (1-Y))
    cost = (1./m) * np.sum(logprobs)
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

# backward propagation
def backward_propagation(X, Y, cache):
    """
    impelement backward propagation 
    
    Arguments: 
    X : training set examples 
    Y : true labels 
    cache : output cache from forward propagation 
    
    Returns: 
    gradients: a dictionary of gradient values.
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) 
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients


In [64]:
# N-D gradient check

# gradient check 
def gradient_check(parameters, gradients, X, Y, epsilon=1e-7):
    """
    Implements gradient check 
    
    Arguments: 
    parameters : dictionary of parameters W, b
    gradients : dictionary of gradients dW, db
    X : input examples 
    Y : true labels 
    Epsilon : nudge value for gradient checking 
    
    Returns: 
    difference : gradient check output
    """
    theta, theta_index = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = theta.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    # compute gradapprox
    for i in range(num_parameters):
        # Compute J_plus
        # for each parameter values, we increase it by epsilon and then calculate the forward
        # propagation, so we will iterate over all values of parameters one by one
        thetaplus = np.copy(theta)
        thetaplus[i][0] = thetaplus[i][0] + epsilon
        J_plus[i], _ = forward_propagation(X, Y, vector_to_dictionary(thetaplus, theta_index, parameters))

        # Compute J_minus
        thetaminus = np.copy(theta)
        thetaminus[i][0] = thetaminus[i][0] - epsilon
        J_minus[i][0], _ = forward_propagation(X, Y, vector_to_dictionary(thetaminus, theta_index, parameters))
        
        # gradappr[i]
        gradapprox[i] = (J_plus[i] - J_minus[i])/ (2*epsilon)
        
    # compute difference 
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator/ denominator
    
    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
       
    return difference
    
    
    
    

In [62]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation(X, Y, parameters)
gradients = backward_propagation(X, Y, cache)
difference = gradient_check(parameters, gradients, X, Y)

Your backward propagation works perfectly fine! difference = 1.1885552035482147e-07
